# ***MEMOIRE MASTER 1 / TRIFFOULLIS-LES-OIES / NEURAL NETWORK***

Ce notebook a été fait sur Google colab, un environnement python en ligne. Certaines commandes (notamment d'interaction avec l'ordinateur ou d'installation de packages) ne fonctionneront donc pas sur un environnement installé en propre sur un ordinateur.

Il a les avantages de rester modulable, tout en permettant aux calculs d'être effectués dans "le cloud" : il n'est limité que par le flux de données et d'instructions à envoyer et recevoir.

## Importation et traitement du fichier `.csv`


### Importation des données

On importe ici les données de `data2011_final.csv` dans une variable `COMMUNES`.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data2011_final.csv to data2011_final.csv


In [ ]:
import pandas as pd

In [ ]:
Dataframe_communes = pd.read_csv("data2011_final.csv", delimiter=";")
len(Dataframe_communes)

36209

In [ ]:
ALL_COMMUNES = pd.DataFrame(Dataframe_communes).to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
COMMUNES, TEST_COMMUNES = train_test_split(ALL_COMMUNES, test_size=0.099, random_state=10)

In [ ]:
COMMUNES

array([['Ligron', 95.0, '72270', ..., 72, 'PL', 2],
       ['Schopperten', 220.0, '67260', ..., 67, 'AL', 3],
       ["Saint-Pierre-d'Autils", 20.0, '27950', ..., 27, 'HN', 2],
       ...,
       ['V\x9cgtlinshoffen', 334.0, '68420', ..., 68, 'AL', 3],
       ['Montmort', 344.0, '71320', ..., 71, 'BO', 3],
       ['Monflanquin', 147.0, '47150', ..., 47, 'AQ', 5]], dtype=object)

In [ ]:
TEST_COMMUNES

array([['Mostuéjouls', 480.0, '12720', ..., 12, 'MP', 5],
       ['La_Villeneuve-lès-Charleville', 210.0, '51120', ..., 51, 'CA',
        3],
       ['Saint-Ouen-des-Alleux', 85.0, '35140', ..., 35, 'BR', 2],
       ...,
       ['Artigues', 640.0, '65100', ..., 65, 'MP', 5],
       ['Vandeléville', 340.0, '54115', ..., 54, 'LO', 3],
       ['Boiry-Saint-Martin', 85.0, '62175', ..., 62, 'NP', 3]],
      dtype=object)

### Appel des bibliothèques nécéssaires

On appelle les bibliothèques qui nous serviront pour la suite.

In [ ]:
import time

In [ ]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Préparation des données

On prépare les données tout comme dans RStudio en les séparant en une base de données test et une base de données d'apprentissage.

In [ ]:
#283 10
random.seed(10)
sample = randint(low = 0, high = len(ALL_COMMUNES), size = 3620)

In [ ]:
TEST_COMMUNES = ALL_COMMUNES[sample[0]]
for i in sample[1:len(sample)]:
    TEST_COMMUNES = vstack((TEST_COMMUNES,ALL_COMMUNES[i]))

COMMUNES = ALL_COMMUNES  # /!\ attention, cela n'est pas vérifé, mais on lie peut être les deux variables entre elles /!\
for i in range(len(sample)):
    COMMUNES = np.delete(COMMUNES,sample[i]-i,0)

KeyboardInterrupt: ignored

In [ ]:
taille = len(COMMUNES)
taille_test = len(TEST_COMMUNES)

print("taille_test : ", taille_test)
print("taille : ", taille)

taille_test :  3585
taille :  32624


In [ ]:
DATACOMMUNES = pd.DataFrame(COMMUNES, columns= Dataframe_communes.columns)
DATATEST_COMMUNES = pd.DataFrame(TEST_COMMUNES,columns= Dataframe_communes.columns)

In [ ]:
DATACOMMUNES

,nom,altitude,country.etc,pop99,longitude,latitude,surface,departement,region,indicatif
0,Ligron,95,72270,358,0.000121196,0.833802,13.67,72,PL,2
1,Schopperten,220,67260,292,0.123051,0.854251,4.42,67,AL,3
2,Saint-Pierre-d'Autils,20,27950,1036,0.0250843,0.857296,7.18,27,HN,2
3,Frocourt,87,60000,514,0.0363853,0.861882,6.49,60,PI,3
4,Vieux-Champagne,130,77370,190,0.0547936,0.8479,8.89,77,IF,1
...,...,...,...,...,...,...,...,...,...,...
32619,Mouettes,130,27220,566,0.0237559,0.853393,8.39,27,HN,2
32620,Vaire-le-Petit,250,25220,197,0.107299,0.825323,1.27,25,FC,3
32621,Vgtlinshoffen,334,68420,478,0.127065,0.838107,4.05,68,AL,3
32622,Montmort,344,71320,183,0.0714228,0.815675,31.71,71,BO,3


In [ ]:
DATACOMMUNES_2 = DATACOMMUNES.sort_values(by = 'departement', ascending = True)
DATATEST_COMMUNES_2 = DATATEST_COMMUNES.sort_values(by = 'departement', ascending = True)

In [ ]:
COMMUNES = DATACOMMUNES_2.to_numpy()
TEST_COMMUNES = DATATEST_COMMUNES_2.to_numpy()

In [ ]:
COMMUNES

array([["Pont-d'Ain", 250.0, 'France', ..., 1, 'RA', 4],
       ['Ambérieux-en-Dombes', 293.0, 'France', ..., 1, 'RA', 4],
       ['Bouligneux', 283.0, 'France', ..., 1, 'RA', 4],
       ...,
       ['Moussy', 90.0, '95640', ..., 95, 'IF', 1],
       ['Puiseux-Pontoise', 100.0, '95650', ..., 95, 'IF', 1],
       ['Pontoise', 62.0, '95300|95000', ..., 95, 'IF', 1]], dtype=object)

In [ ]:
TEST_COMMUNES

array([['Cruzilles-lès-Mépillat', 215.0, 'France', ..., 1, 'RA', 4],
       ['Saint-Laurent-sur-Saône', 173.0, 'France', ..., 1, 'RA', 4],
       ['Biziat', 210.0, 'France', ..., 1, 'RA', 4],
       ...,
       ['Viarmes', 50.0, '95270', ..., 95, 'IF', 1],
       ['Le_Bellay-en-Vexin', 114.0, '95750', ..., 95, 'IF', 1],
       ['Arronville', 90.0, '95810', ..., 95, 'IF', 1]], dtype=object)

In [ ]:
doublons = zeros(taille_test)
for i in range(taille_test):
    doublons[i] = TEST_COMMUNES[i,0] in COMMUNES[:,0]

sum(doublons)

400.0

In [ ]:
TEST_COMMUNES[1,0] in COMMUNES[:,0]

False

## Motivations

On va ici s'intéresser à localiser les communes françaises à l'aide de réseaux de neurones. Un notebook écrit en L3 avec David Gontier rentre dans le détail de la théorie mathématique des MLP pour de la classification. Peut être faudra-t-il le résumer dans le rapport final ?

## Méthodes envisagées

L'idée est ici de faire "lire" l'ordinateur, ou plutôt de lui faire se représenter les noms des villes par leur proximité dans leur écriture. Plusieurs solutions sont alors envisagées :

### *1. Proximité des noms deux à deux*

Il s'agit ici d'utiliser d'une méthode qui va "calculer" la proximité des noms des villes deux à deux.

En effet, on va utiliser des réseaux de neurones "classiques" prenant en entrée et en sortie des nombres. Il s'agit donc pour chaque commune de transformer la chaine de caractères en un vecteur, qui sera alors compréhensible par l'ordinateur.

L'approche assez "brute et violente" consisterait à calculer les rapport de proximité toponymiques pour toutes les villes, entre elles, deux à deux.
Mais avec la taille de la base de donnée, cela va revenir à un input dans $\mathbb{R}^{35289}$, dimension assez conséquente.

On va alors raisonner par département, prendre un nom de référence pour chaque département (celui ayant le plus de proximité avec tous les autres, le plus central, ou alors, choisi arbitrairement).
On aura ainsi, pour chaque communes, la proximité de son nom à toutes les communes de référence choisies.

Par exemple, dans le cadre des départements, on en compte $95$, cela nous donne donc un input dans $\mathbb{R}^{95}$ et un output, soit dans $\mathbb{R}^2$ (ses coordonnées géographiques) soit dans $\mathbb{N}$ tronqué (son département, ou sa région).

***On explore ici plus bas le cas des départements, avec la commune de référence choisie au hasard, avec sortie par classification pour les départements.***

On comprends alors que si la proximité géographique est corrélée à la proximité toponymique, on espère avoir des résultats satisfaisants sur les sorties.

### *2. Word Embedding*

Il s'agit dans ce cas d'apprendre "réellement à lire à l'ordinateur". Un classique du *Word Embedding* est de passer par des modèles d'apprentissage de type `Word2Vec` qui prennent en entrée une liste de caractère et apprennent par rapport à une sortie donnée, ici aussi possiblement dans $\mathbb{R}^2$ ou bien $\mathbb{N}$ en fonction de l'output voulu.


## 1.Proximité des noms deux à deux

#### `fuzzywuzzy` library et similarité entre les noms

On utilise dans un premier temps la librarie `fuzzy wuzzy` qui permet de calculer la similarité entre deux mots ou groupe de mots.

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     |████████████████████████████████| 51kB 1.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144678 sha256=8f6e6d1ff7f0c4ec124d4d0672df91ce22736545bd19941f5d432c1e04b2a4e4
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


In [ ]:
from fuzzywuzzy import fuzz

On va alors regarder la théorie derrière cette librarie.

`fuzzywuzzy`, malgré son nom qui peut paraître drôle, est bien intéréssante. Elle se base principalement sur la *distance de Leveshtein*, une métrique pour les chaines de caractères qui permet de mesurer les différences entre les mots.

Elle est définie de la manière suivante :


---


La distance de Levenshtein entre deux chaines de caractères $a, b$ (de taille $|a|$ et $|b|$ respectives) est donnée par $\operatorname{lev}_{a,b}(|a|,|b|)$, où :

$$\qquad\operatorname{lev}_{a,b}(i,j) = \begin{cases}
  \max(i,j) & \text{ si } \min(i,j)=0, \\
  \min \begin{cases}
          \operatorname{lev}_{a,b}(i-1,j) + 1 \\
          \operatorname{lev}_{a,b}(i,j-1) + 1 \\
          \operatorname{lev}_{a,b}(i-1,j-1) + 1_{(a_i \neq b_j)}
       \end{cases} & \text{ autrement.}
\end{cases}$$

où  $1_{(a_i \neq b_j)}$ est la fonction indicatrice et $\operatorname{lev}_{a,b}(i,j)$ est la distance entre le premier $i$ème caractère de $a$ et le premier $j$ème caractère de $b$. $i$ and $j$ are 1-based indices.


---


En "français dans le texte", elle est égale au nombre minimal de caractères qu'il faut supprimer, insérer ou remplacer pour passer d’une chaîne à l’autre.

On va nous s'intéresser au ratio de similarité, donné lui par :

$$\frac{(|a| + |b|) - {lev}_{a,b}(i,j)}{|a| + |b|}$$

Ce ratio (correspondant à la fonction `fuzz.ration`peut être retrouvé avec d'autre libraries, mais nous allons utiliser celle-ci car elle permet d'aller plus loin, notamment avec des fonctions telles que `fuzz.partial_ratio()` ou encore `fuzz.token_sort_ratio()` qui permettent d'améliorer les résultats dans certaines situations, ou en tout cas de constater une silimilitude d'une manière différence. Il sera alors intéréssant de comparer les résultats des différents réseaux de neurones avec ces métriques différentes.

#### Création de la matrice de proximité des noms **/!\ opération très couteuse et non nécéssaire pour la classification /!\**

On crée ici une matrice de $32589$ par $32589$ (prends environ 30 minutes) donnant pour chaque commune sa similarité dans son nom par rapport à toutes les autres.

In [ ]:
Data_levenshtein = array([[fuzz.ratio(x,y) for x in COMMUNES[:,0]] for y in COMMUNES[:,0]])

In [ ]:
Data_levenshtein

Pour des soucis de temps de calcul, on enregistre ensuite dans un dossier la matrice obtenue. (qu'on peut ensuite réutiliser par la suite juste en important le dossier).

In [ ]:
numpy.savetxt("levenshtein_ratios.csv", Data_levenshtein, delimiter=";")

In [ ]:
files.download("levenshtein_ratios.csv")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 54046, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

*Code pour importer la matrice avec les ratios levenshtein :*

In [ ]:
uploaded = files.upload()

Saving levenshtein_triangulaire.csv to levenshtein_triangulaire.csv


In [ ]:
Frame_leveshtein = pd.read_csv("levenshtein_triangulaire.csv", delimiter=",")
Data_fuzz_communes = pd.DataFrame(Frame_leveshtein).to_numpy()
Data_fuzz_communes
Frame_leveshtein

,1.000000000000000000e+02,6.800000000000000000e+01,4.000000000000000000e+01,4.300000000000000000e+01,3.300000000000000000e+01,3.200000000000000000e+01,2.600000000000000000e+01,4.100000000000000000e+01,3.100000000000000000e+01,3.900000000000000000e+01,2.900000000000000000e+01,2.700000000000000000e+01,3.100000000000000000e+01.1,2.700000000000000000e+01.1,2.900000000000000000e+01.1,2.900000000000000000e+01.2,2.600000000000000000e+01.1,3.200000000000000000e+01.1,2.400000000000000000e+01,3.200000000000000000e+01.2,2.200000000000000000e+01,2.700000000000000000e+01.2,1.400000000000000000e+01,2.000000000000000000e+01,1.900000000000000000e+01,2.400000000000000000e+01.1,1.900000000000000000e+01.1,3.000000000000000000e+01,2.100000000000000000e+01,1.800000000000000000e+01,3.500000000000000000e+01,1.900000000000000000e+01.2,1.500000000000000000e+01,1.500000000000000000e+01.1,3.200000000000000000e+01.3,2.700000000000000000e+01.3,8.000000000000000000e+00,2.000000000000000000e+01.1,2.700000000000000000e+01.4,1.300000000000000000e+01,...,2.800000000000000000e+01.1070,1.900000000000000000e+01.2670,2.700000000000000000e+01.1545,3.000000000000000000e+01.1060,2.100000000000000000e+01.2125,2.200000000000000000e+01.897,2.300000000000000000e+01.961,2.200000000000000000e+01.898,3.300000000000000000e+01.789,3.300000000000000000e+01.790,2.800000000000000000e+01.1071,2.900000000000000000e+01.1769,2.200000000000000000e+01.899,2.900000000000000000e+01.1770,2.400000000000000000e+01.2379,3.000000000000000000e+01.1061,2.000000000000000000e+01.1831,2.600000000000000000e+01.1968,2.500000000000000000e+01.1362,2.900000000000000000e+01.1771,2.700000000000000000e+01.1546,1.800000000000000000e+01.1163,2.000000000000000000e+01.1832,2.300000000000000000e+01.962,1.900000000000000000e+01.2671,2.400000000000000000e+01.2380,1.700000000000000000e+01.336,2.400000000000000000e+01.2381,2.000000000000000000e+01.1833,2.100000000000000000e+01.2126,1.900000000000000000e+01.2672,2.000000000000000000e+01.1834,1.400000000000000000e+01.2033,1.400000000000000000e+01.2034,1.900000000000000000e+01.2673,2.900000000000000000e+01.1772,2.200000000000000000e+01.900,2.600000000000000000e+01.1969,3.200000000000000000e+01.953,2.400000000000000000e+01.2382
0,0.0,100.0,47.0,40.0,21.0,41.0,28.0,38.0,27.0,41.0,23.0,36.0,33.0,36.0,23.0,23.0,28.0,48.0,36.0,28.0,29.0,29.0,8.0,14.0,21.0,32.0,20.0,40.0,22.0,19.0,32.0,14.0,16.0,8.0,14.0,21.0,17.0,21.0,7.0,14.0,...,34.0,21.0,21.0,32.0,22.0,23.0,24.0,28.0,29.0,34.0,29.0,36.0,33.0,30.0,26.0,19.0,7.0,21.0,20.0,25.0,23.0,19.0,29.0,12.0,13.0,26.0,18.0,25.0,21.0,22.0,14.0,29.0,23.0,15.0,21.0,36.0,29.0,33.0,33.0,30.0
1,0.0,0.0,100.0,72.0,42.0,48.0,40.0,30.0,31.0,32.0,27.0,25.0,38.0,42.0,27.0,27.0,31.0,32.0,40.0,24.0,27.0,32.0,9.0,17.0,24.0,30.0,31.0,34.0,26.0,22.0,29.0,24.0,29.0,19.0,24.0,17.0,30.0,25.0,8.0,33.0,...,27.0,16.0,25.0,47.0,36.0,26.0,21.0,26.0,27.0,32.0,27.0,29.0,31.0,39.0,22.0,22.0,8.0,16.0,23.0,29.0,31.0,21.0,25.0,34.0,23.0,15.0,13.0,21.0,8.0,17.0,24.0,17.0,27.0,28.0,24.0,40.0,20.0,44.0,38.0,28.0
2,0.0,0.0,0.0,100.0,38.0,37.0,44.0,34.0,21.0,30.0,25.0,31.0,36.0,31.0,33.0,33.0,41.0,37.0,38.0,22.0,25.0,30.0,8.0,23.0,22.0,21.0,36.0,33.0,16.0,21.0,28.0,37.0,17.0,17.0,22.0,15.0,9.0,31.0,8.0,38.0,...,26.0,22.0,31.0,50.0,46.0,18.0,32.0,29.0,21.0,36.0,31.0,32.0,29.0,37.0,21.0,21.0,23.0,15.0,29.0,27.0,29.0,27.0,15.0,32.0,21.0,7.0,12.0,20.0,8.0,24.0,22.0,31.0,17.0,32.0,22.0,43.0,25.0,35.0,35.0,21.0
3,0.0,0.0,0.0,0.0,100.0,67.0,40.0,26.0,38.0,53.0,33.0,29.0,38.0,43.0,17.0,33.0,27.0,27.0,16.0,27.0,10.0,10.0,33.0,14.0,27.0,0.0,25.0,13.0,15.0,24.0,25.0,40.0,36.0,55.0,13.0,14.0,0.0,14.0,14.0,0.0,...,7.0,27.0,14.0,8.0,17.0,19.0,21.0,7.0,7.0,7.0,10.0,16.0,7.0,23.0,12.0,12.0,14.0,13.0,12.0,11.0,21.0,11.0,14.0,32.0,12.0,12.0,20.0,22.0,14.0,15.0,13.0,14.0,17.0,15.0,40.0,16.0,20.0,9.0,9.0,8.0
4,0.0,0.0,0.0,0.0,0.0,100.0,50.0,33.0,24.0,50.0,46.0,53.0,35.0,53.0,31.0,31.0,35.0,38.0,23.0,38.0,10.0,0.0,15.0,13.0,25.0,22.0,12.0,12.0,14.0,11.0,24.0,25.0,33.0,33.0,25.0,13.0,18.

In [ ]:
Data_levenshtein = Data_fuzz_communes.transpose() + Data_fuzz_communes
for i in range(taille):
    Data_levenshtein[i,i] = Data_levenshtein[i,i]/2

#### Compte des communes par département

On va ensuite compter le nombre de communes par département pour :

*   pouvoir en choisir une aléatoirement comme "référence" comme expliqué plus haut.
*   effectuer des statistiques descriptives



In [ ]:
compte_dpt = zeros(95)
for i in range(95):
    compte_dpt[i] = np.sum(COMMUNES[:,7] == i+1)

compte_dpt_croissant_zeros = zeros(96)
for i in range(95):
    compte_dpt_croissant_zeros[i+1] = sum(compte_dpt[0:(i+1)])

compte_dpt_croissant = compte_dpt_croissant_zeros[1:len(compte_dpt_croissant_zeros)]

In [ ]:
print("compte communes par département :")
print(compte_dpt)
print("compte croissant des communes par département :")
print(compte_dpt_croissant)
print("compte croissant des communes par département avec 0 devant :")
print(compte_dpt_croissant_zeros)

compte communes par département :
[384. 746. 284. 185. 163. 150. 313. 396. 297. 384. 392. 268. 109. 639.
 242. 367. 436. 262. 258.   0. 622. 345. 242. 496. 542. 331. 612. 363.
 257. 325. 524. 413. 494. 311. 316. 231. 240. 485. 489. 293. 266. 297.
 237. 202. 303. 298. 282. 163. 321. 530. 569. 395. 227. 533. 443. 235.
 673. 277. 596. 629. 458. 794. 425. 493. 427. 207. 473. 349. 253. 482.
 511. 337. 277. 270.   1. 665. 466. 236. 278. 695. 291. 178. 138. 132.
 255. 256. 176. 472. 412.  88. 166.  33.  37.  44. 167.]
compte croissant des communes par département :
[  384.  1130.  1414.  1599.  1762.  1912.  2225.  2621.  2918.  3302.
  3694.  3962.  4071.  4710.  4952.  5319.  5755.  6017.  6275.  6275.
  6897.  7242.  7484.  7980.  8522.  8853.  9465.  9828. 10085. 10410.
 10934. 11347. 11841. 12152. 12468. 12699. 12939. 13424. 13913. 14206.
 14472. 14769. 15006. 15208. 15511. 15809. 16091. 16254. 16575. 17105.
 17674. 18069. 18296. 18829. 19272. 19507. 20180. 20457. 21053. 21682.
 22140. 2

Le 0 en 20ème position correspond à la Corse que l'on a retiré de notre étude, ainsi que le 1 pour le 75 qui correspond uniquement à la ville de Paris.

#### Determination de la commune la plus représentative pour chaque département

In [ ]:
#crée une matrice 32589 par 3 avec pour chaque commune :
# - la moyenne de sa métrique dans son département
# - sa propre métrique de 100.
# - le nombre de communes dans son département

Communes_moyenne_ds_dpt = zeros((taille,2))
for i in range(taille):
    dpt_i = COMMUNES[i,7]
    b_inf = int(compte_dpt_croissant_zeros[dpt_i-1])
    b_sup = int(compte_dpt_croissant_zeros[dpt_i])
    Communes_moyenne_ds_dpt[i,0] = mean(Data_levenshtein[i][b_inf:b_sup])
    Communes_moyenne_ds_dpt[i,1] = compte_dpt[COMMUNES[i,7]]

#crée un vecteur avec pour chaque commune la moyenne de sa métrique dans son département sans se compter.

Communes_scoremoy_dpt = zeros(taille)
for i in range(taille):
    Xbarre = Communes_moyenne_ds_dpt[i,0]
    n = Communes_moyenne_ds_dpt[i,1]
    Communes_scoremoy_dpt[i] = (Xbarre - (float(100))/n)*(n/(n-1))

#retourne un vecteur avec les indexations des 95 communes ayant la meilleure métrique dans son département.

best_dpt = zeros(95)
for i in range(95):
    best_dpt[i] = argmax(Communes_scoremoy_dpt[int(compte_dpt_croissant_zeros[i]),int(compte_dpt_croissant_zeros[i+1])])

IndexError: ignored

In [ ]:
numpy.savetxt("best_dpt.csv", best_dpt, delimiter=";")
files.download("best_dpt.csv")

32589

In [ ]:
best_dpt

In [ ]:
numpy.savetxt("best_dpt.csv", best_dpt, delimiter=";")
files.download("best_dpt.csv")

#### Statistiques descriptives sur la base d'apprentissage **/!\ ne pas run car utilise la très grosse matrice /!\**

Pour les statistiques descriptives :

In [ ]:
sim_dpt = zeros((taille,95))
for i in range(taille):
    for j in range(95):
        sim_dpt[i,j] = mean(Data_levenshtein[i][int(compte_dpt_croissant_zeros[j]):int(compte_dpt_croissant_zeros[j+1])])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### Tirage aléatoire d'une commune de référence par département

On va alors tirer aléatoirement une commune dans chaque département pour s'en servir comme référence : 

In [ ]:
#283 10

random.seed(10)

alea_dpt = zeros(95)
for i in range(95):
    if i != 19 : #probleme de la corse
        alea_dpt[i] = randint(low = 0, high = compte_dpt[i]) #de 0 a max-1 pour correspondre à l'indexation sous python

numeros_communes_ref_alea = zeros(95)
numeros_communes_ref_alea[0] = alea_dpt[0]
for i in range(1,95):
    numeros_communes_ref_alea[i] = alea_dpt[i] + compte_dpt_croissant[i-1]

In [ ]:
#vérification que la ville selectionnée pour le 75 est bien Paris
COMMUNES[int(numeros_communes_ref_alea[74]),0]

'Paris'

#### MLP avec input vis à vis de la référence et classification par département : `MLP_indpt_outdpt`

On construit ensuite la matrice qui servira d'entrée, et celle qui servira de sortie (on choisit ici un output de type classification sur les départements dans un premier tems) :

In [ ]:
INPUT_alea = np.zeros((taille,95))
for i in range(taille):
    for j in range(95):
        INPUT_alea[i][j] = fuzz.ratio(COMMUNES[i,0],COMMUNES[int(numeros_communes_ref_alea[j]),0])

OUTPUT = COMMUNES[:,7].astype('int')

In [ ]:
print('INPUT_alea :')
print(INPUT_alea)
print('size INPUT_alea : ',len(INPUT_alea), ' x ', len(INPUT_alea[0]))

INPUT_alea :
[[40. 21. 25. ... 38. 25. 15.]
 [21. 42. 16. ... 40. 24. 28.]
 [50. 34. 38. ... 29. 25. 30.]
 ...
 [38. 16. 33. ... 12. 33. 26.]
 [38. 46. 27. ... 37. 36. 48.]
 [56. 30. 29. ... 42. 43. 24.]]
size INPUT_alea :  32624  x  95


In [ ]:
print('OUTPUT :')
print(OUTPUT)
print('size OUTPUT : ',len(OUTPUT))

OUTPUT :
[ 1  1  1 ... 95 95 95]
size OUTPUT :  32624


On adapte ensuite la base de test pour l'entrer dans le réseau :

In [ ]:
INPUT_TEST = np.zeros((taille_test,95))
for i in range(taille_test):
    for j in range(95):
        INPUT_TEST[i][j] = fuzz.ratio(TEST_COMMUNES[i,0],COMMUNES[int(numeros_communes_ref_alea[j]),0])

On implémente alors le réseau de neurones, ici un MLP : 

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
MLP_indpt_outdpt = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(180, 180*4), random_state=10, max_iter = 400)

In [ ]:
MLP_indpt_outdpt.fit(INPUT_alea,OUTPUT)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(180, 720), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=400,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
predict = MLP_indpt_outdpt.predict(INPUT_TEST)

In [ ]:
predict

array([74, 38, 10, ..., 91,  7, 68])

In [ ]:
MLP_indpt_outdpt.predict("Triffouillis-Les-Oies")

ValueError: ignored

In [ ]:
REAL_OUTPUT_TEST = TEST_COMMUNES[:,7].astype('int')
REAL_OUTPUT_TEST

array([47, 71, 68, ..., 77, 57, 89])

In [ ]:
#cette fonction retourne le nombre de Vrai et de Faux résultats entre la prédiction et la base de test.
#elle retourne une liste avec le nobre de vrai, le nombre de faux, et le pourcentage de vrais.

def VF_predict(prediction,real_output):
    l = len(prediction)
    V = 0
    F = 0
    for i in range(l):
        if prediction[i] == real_output[i]:
            V += 1
        else :
            F += 1
    percent = V/l
    return(V,F,percent)

In [ ]:
VF_predict(predict,REAL_OUTPUT_TEST)

(46, 3574, 0.01270718232044199)

In [ ]:
#cette fonction teste la performance des différents "ratio" possibles entre le nombre de couches et la taille des couches du MLP.

def compar_MLP_ratio(IN, OUT, IN_test,real_out, start_layer_size = 50, nb_iter = 5, max_iter = 300):
    prediction_percent = zeros(n)
    for k in range(n):
        MLP = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(start_layer_size, start_layer_size*(k+1)), random_state=10, max_iter = max_iter)
        MLP.fit(IN,OUT)
        prediction_percent[k] = VF_predict(MLP.predict(IN_test),real_out)[2]
        print(VF_predict(MLP.predict(IN_test),real_out))
    return prediction_percent

In [ ]:
compar_ratio = compar_MLP_ratio(INPUT_alea, OUTPUT, INPUT_TEST, REAL_OUTPUT_TEST)

NameError: ignored

In [ ]:
compar_ratio

array([0.10828729, 0.13314917, 0.16657459, 0.07486188, 0.04696133,
       0.14254144, 0.0301105 , 0.10082873, 0.03895028, 0.05662983])

In [ ]:
#cette fonction teste la performance des différentes tailles de réseaux de neurones (pour un ratio -comme définit ci dessus- fixé).

def compar_MLP_size(IN,OUT, IN_test,real_out, n = 4, start_layer_size = 30, nb_iter = 10, max_iter = 300):
    prediction_percent = zeros(nb_iter)
    for k in range(nb_iter):
        MLP = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(start_layer_size*(k+1), start_layer_size*(k+1)*n), random_state=10, max_iter = max_iter)
        MLP.fit(IN,OUT)
        prediction_percent[k] = VF_predict(MLP.predict(IN_test),real_out)[2]
        print("pour MLP size (", start_layer_size*(k+1), ",", start_layer_size*(k+1)*n, ") : ", VF_predict(MLP.predict(IN_test),real_out))
    return prediction_percent

In [ ]:
compar_size = compar_MLP_size(INPUT_alea, OUTPUT, INPUT_TEST, REAL_OUTPUT_TEST)

In [ ]:
plt.plot(array([30,60,90,120,150,180,210,240,270,300]),compar_size)
plt.ylabel("pourcentage d'exactitude")
plt.xlabel("nombre de couches (4*x) et taille de couches (x) utilisées")
plt.axis([0, 310, 0, 1])
plt.show()

Algorythme du MLP avec les paramètres optimaux comme ils ont pu être testés ci-dessus :

In [ ]:
def optimized_MLP(IN,OUT, IN_test,real_out, layer_size = 180, nb_layers = 180*4, max_iter = 400):
    MLP = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(layer_size, nb_layers, random_state=10, max_iter = max_iter)
    MLP.fit(IN,OUT)
    prediction = VF_predict(MLP.predict(IN_test),real_out)
    print("optimized MLP returns (", layer size, ",", nb_layers, ") : ", prediction)
    return(prediction)

SyntaxError: ignored

In [ ]:
Base_aleatoire = optimized_MLP(INPUT_alea, OUTPUT, INPUT_TEST, REAL_OUTPUT_TEST)
#Base_best = optimized_MLP(INPUT_best, OUTPUT, INPUT_TEST, REAL_OUTPUT_TEST)

NameError: ignored

In [ ]:
A = zeros((3,3))
A[0] = ones(3)
A[1] = 2*ones(3)

A = array([k*ones(10) for k in range(10)])

sample = array([0, 1, 8])

A_bis = A
for i in range(len(sample)):
    A_bis = np.delete(A,sample[i]-i,0)

print(A,A_bis)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]] [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]]


In [ ]:
for i in range(len(sample)):
    COMMUNES_del = np.delete(COMMUNES,sample[i]-i,0)
    COMMUNES = COMMUNES_del

On va ici regarder si des reseaux de neurones non-supervisés peuvent s'avérer pertinents. Le seul réseau de neurones non-supervisé disponible avec la bibliothèque sickit-learn est une machine de Boltzman de type Bernouilli : elle ne prend en entrée que des valeures binaires, ou entre 0 et 1. Cela tombe bien, 